In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
import pickle
import pandas as pd

In [39]:
# MongoDB에서 데이터 가져오기
client = MongoClient('mongodb://192.168.0.50:27017/')
db = client["DB_SGMN"]
collection = db["COL_SCRAPPING_TOSS_COMMENT_HISTORY"]

In [40]:
# 임의로 100개 레코드 가져오기
data = list(collection.find().limit(30000))
df = pd.DataFrame(data)


In [41]:
# 텍스트 컬럼 선택
texts = df['COMMENT'].astype(str).tolist()


In [42]:
# TF-IDF 벡터화
tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit(texts)
result_vectors = tfidfVectorizer.transform(texts)

# 문장 간 유사도 측정
df_similarity = cosine_similarity(result_vectors, result_vectors)
df_similarity_matrix = pd.DataFrame(
    data=df_similarity,
    index=texts,
    columns=texts
)

# 새로운 문장 유사도 계산
new_sentence = "사이버 보안은 디지털 시대의 중요한 이슈로 부상하고 있으며, 지속적인 관심과 개선이 요구된다."
new_tfidf_vector = tfidfVectorizer.transform([new_sentence])
similarity_array = cosine_similarity(new_tfidf_vector, result_vectors).flatten()

argmax_similarity = similarity_array.argmax()
most_similar_sentence = texts[argmax_similarity]
most_similar_score = similarity_array[argmax_similarity]

# 출력
print("\n[문장 간 유사도 매트릭스]")
print(df_similarity_matrix)

print("\n[새로운 문장 유사도 결과]")
print("새로운 문장:", new_sentence)
print("가장 유사한 문장:", most_similar_sentence)
print(f"유사도 점수: {most_similar_score:.4f}")


[문장 간 유사도 매트릭스]
                                                    머선일이야  뭐냐 너..왜 희망 갖게하냐  \
머선일이야                                                 1.0              0.0   
뭐냐 너..왜 희망 갖게하냐                                       0.0              1.0   
몇개월 기다리다가 조금 올라서 뺏더니 잘 올라가네.. ㅠ                       0.0              0.0   
얘는 얼마까지 갈까요                                           0.0              0.0   
애플, 2026년 출시 목표로 시리 전면 개편 추진\n\n(주요내용)\n1. 대규모 ...    0.0              0.0   
...                                                   ...              ...   
간다는 사람들은 정확한 근거가 있는 거임?                               0.0              0.0   
일단 주워놓고 19만되면 팔아야징. 20만까지도 될 거 같은데 안전하게!              0.0              0.0   
ㅋㅋㅋㅋㅋㅋㅋㅋㅋ내년즈음 봅세다                                     0.0              0.0   
20닉스 가려나...                                           0.0              0.0   
닉스닉스 가자가자가자규                                          0.0              0.0   

                                              

In [43]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(
    max_features=500,  # 최대 500개의 단어로 제한
    stop_words=['것', '이', '그', '저', '수', '등', '은', '위'],  # 불용어 제거
)
tfidf_matrix = vectorizer.fit_transform(texts)

In [44]:
# Pickle로 TF-IDF 모델과 데이터 저장
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

print("TF-IDF 모델과 데이터를 Pickle로 저장했습니다.")

TF-IDF 모델과 데이터를 Pickle로 저장했습니다.
